## Reading the Dataset

In [1]:
from os.path import join
import pandas as pd
import csv

with open(join("data","iranian_users.csv")) as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    users = {row[2]: {'row': row, 'tweets': [], 'scores': None} for row in csvreader if row[0] != 'userid'}

tweet_counter = 0
missing_users = set([])
with open(join("data","iranian_tweets.csv")) as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for tweet in csvreader:
        screen_name = tweet[3]
        if screen_name not in users:
            missing_users.add(screen_name)
        else:
            users[screen_name]['tweets'].append(tweet)
            tweet_counter += 1

missing_users.remove('user_screen_name')
if missing_users:
    print("Missing users with tweets: " + ', '.join(list(not_in_sources)))
    print()

print("Loaded {tn} tweets from {un} users".format(tn=tweet_counter, un=len(users)))

Loaded 1122936 tweets from 770 users


## Utility Functions

In [2]:
from datetime import datetime, timedelta
import json
from urllib.parse import urlparse


DATASET_DATE = datetime(2018, 9, 1)

# Load the Media Bias/Fact Check db
with open(join('data', 'sources.json')) as infile:
    sources = json.load(infile)

FACTUAL_HALF = set(['MIXED'])
FACTUAL_COMPLETELY = set(['HIGH', 'VERY HIGH'])

# A failed attemp to write a function to decode the reported_location field
# import pycountry
# _countries_cache = {}
# def get_country(name):
#     c = _countries_cache.get(name, None)
#     if name not in _countries_cache:
#         c = pycountry.countries.get(name=name)
#         if not c and len(name) == 2:
#             c = pycountry.countries.get(alpha_2=name.upper())
#         if not c and len(name) == 3:
#             c = pycountry.countries.get(alpha_3=name.upper())
#         if not c:
#             c = pycountry.countries.get(official_name=name)
#         _countries_cache[name] = c
#         if not country:
#             print("failed to find country for " + name)
#     return c

## Calculating the Scores for each User

In [20]:
not_in_sources = set([])
dataset_sources_factuality = {}


users_with_no_tweet = set([])
max_account_age = None
min_account_age = None

user_counter = 0
for screen_name, profile in users.items():
    n = len(profile['tweets'])

    interface_language = profile['row'][9].lower()[:2]
    timezone = profile['row'][9].lower()[:2]
    account_age = (DATASET_DATE - datetime.strptime(profile['row'][8], '%Y-%m-%d')).days
    if not max_account_age or max_account_age < account_age:
        max_account_age = account_age
    if not min_account_age or min_account_age > account_age:
        min_account_age = account_age
#     reported_location = profile['row'][3]
#     if reported_location:
#         reported_location = reported_location.split(',')[-1].strip()
#         country = get_country(reported_location)

    if n == 0:
        users_with_no_tweet.add(screen_name)
        profile['scores'] = None
        continue

    language_score_unmatched = 0
    tweet_time_bin = [0] * 24
    linked_to_not_factual_domain = 0
    linked_to_known_domain = 0
    for tweet in profile['tweets']:
        if tweet[11][:2].lower() != interface_language:
            language_score_unmatched += 1

        tweet_time = datetime.strptime(tweet[13], '%Y-%m-%d %H:%M')
        tweet_time_bin[tweet_time.hour] += 1

        urls = tweet[-3][2:-2].split()
        for url in urls:
            domain = urlparse(url).netloc.lower()
            if domain not in sources and domain.count('.') > 1:
                domain = domain[domain.index('.')+1:]
            if domain not in sources:
                not_in_sources.add(domain)
                continue
            linked_to_known_domain += 1
            factual = sources[domain][0]['factual']
            if factual not in dataset_sources_factuality:
                dataset_sources_factuality[factual] = 1
            else:
                dataset_sources_factuality[factual] += 1
            if factual in FACTUAL_HALF:
                linked_to_not_factual_domain += 0.5
            elif factual not in FACTUAL_COMPLETELY:
                linked_to_not_factual_domain += 1

    max_sliding_time_window_sum = 0
    for i in range(24):
        sliding_time_window_sum = sum(tweet_time_bin[i:(i+8) % 24])
        if sliding_time_window_sum > max_sliding_time_window_sum:
            max_sliding_time_window_sum = sliding_time_window_sum

    # Normalization
    language_score = language_score_unmatched / n
    working_time_score = ((max_sliding_time_window_sum / n) - (1/3)) * 3 / 2
    if linked_to_known_domain > 0:
        linked_to_domains_score = linked_to_not_factual_domain / linked_to_known_domain
    else:
        linked_to_domains_score = 0

    profile['scores'] = [
        account_age,  # To be normalized
        language_score,
        working_time_score,
        linked_to_domains_score
    ]

    if user_counter == 64:
        break
    user_counter += 1

user_counter = 0
for screen_name, profile in users.items():
    if profile['scores']:
        if max_account_age and min_account_age and max_account_age > min_account_age:
            profile['scores'][0] = (profile['scores'][0] - min_account_age) / (max_account_age - min_account_age)

        score = sum(profile['scores'])
        print('{name:16}: {score:6.2f} = ({vector})'.format(
            name=screen_name[:16], score=score, vector=', '.join(['{0:3.1f}'.format(v) for v in profile['scores']])))
        if user_counter == 64:
            break
        user_counter += 1

print()
print("{num} users are suspended without having any tweets in the dataset: {list}".format(
    num=len(users_with_no_tweet), list=', '.join([s[:16] for s in users_with_no_tweet])))
print()
print("Domains not in the Media Bias/Fact Check db: " + ', '.join(list(not_in_sources)))
print()
print("Overall factuality of the links in the tweets:" + str(dataset_sources_factuality))


57e2082d64baa89d:   1.61 = (0.1, 1.0, 0.5, 0.0)
11891c406c088fdd:   1.55 = (0.5, 0.1, 1.0, 0.0)
5ddbd530097789a4:   0.96 = (0.1, 0.2, 0.4, 0.2)
50012d5e4f959a3d:   1.66 = (0.3, 0.2, 0.5, 0.8)
28478f20c217a672:   1.86 = (0.1, 1.0, 0.8, 0.0)
260bbf8c9ca24c63:   1.71 = (0.1, 1.0, 0.6, 0.0)
3f1a40fa0636db86:   1.47 = (0.1, 1.0, 0.4, 0.0)
bc2adb39c920650d:   1.89 = (0.1, 1.0, 0.7, 0.1)
fd01dd625797c4f4:   1.66 = (0.1, 1.0, 0.6, 0.0)
2d761afd8c25c25e:   1.60 = (0.1, 1.0, 0.5, 0.0)
f3776839f137b8fd:   1.92 = (0.3, 1.0, 0.6, 0.0)
c2577510f436a286:   1.69 = (0.1, 1.0, 0.6, 0.0)
0669e7730d2cacb6:   1.60 = (0.0, 1.0, 0.6, 0.0)
67179a5165a72056:   2.21 = (0.2, 1.0, 1.0, 0.0)
13af1122b942a674:   2.04 = (0.1, 1.0, 1.0, 0.0)
freesaudiawomen :   1.65 = (0.3, 1.0, 0.4, 0.0)
3c536b8d54ca334c:   1.37 = (0.1, 1.0, 0.3, 0.0)
aa757d486c6d8bfc:   1.99 = (0.2, 1.0, 0.8, 0.0)
ab2adf87b21afa43:   1.55 = (0.1, 0.3, 1.0, 0.2)
0a00d0bfdd68bf53:   2.03 = (0.3, 1.0, 0.7, 0.0)
a3d8eccd0b8b2cec:   1.98 = (0.2, 1.0, 0.

## Experiment: Clustering

In [21]:
from sklearn.cluster import KMeans

labels = {}

kmeans = KMeans(n_clusters=4)
kmeans.fit([profile['scores'] for profile in users.values() if profile['scores']])
for profile in users.values():
    if profile['scores']:
        label = int(kmeans.predict([profile['scores']]))
        if label not in labels:
            labels[label] = [profile]
        else:
            labels[label].append(profile)

for label, profiles in labels.items():
    print("{num} users in cluser {label}:".format(num=len(profiles), label=label))
    print('\n'.join(['{name:16}: {vector}'.format(name=p['row'][2][:16], vector=' '.join(['{0:3.1f}'.format(v) for v in p['scores']])) for p in profiles]))
    print()


21 users in cluser 0:
57e2082d64baa89d: 0.1 1.0 0.5 0.0
260bbf8c9ca24c63: 0.1 1.0 0.6 0.0
3f1a40fa0636db86: 0.1 1.0 0.4 0.0
fd01dd625797c4f4: 0.1 1.0 0.6 0.0
2d761afd8c25c25e: 0.1 1.0 0.5 0.0
f3776839f137b8fd: 0.3 1.0 0.6 0.0
c2577510f436a286: 0.1 1.0 0.6 0.0
0669e7730d2cacb6: 0.0 1.0 0.6 0.0
freesaudiawomen : 0.3 1.0 0.4 0.0
3c536b8d54ca334c: 0.1 1.0 0.3 0.0
254403684f5a792b: 0.2 1.0 0.5 0.0
507cb5a10918355e: 0.1 1.0 0.6 0.0
alsaudiatimes   : 0.3 1.0 0.2 0.2
fa3140e3b9b4af67: 0.3 1.0 0.4 0.0
eda1fe260cc6695d: 0.1 1.0 0.6 0.0
374dff7f2f8b3484: 0.1 0.8 0.4 0.0
5dc3400ed880dd73: 0.2 1.0 0.3 0.0
1f33d9e4bebd609c: 0.4 0.9 0.5 0.0
02a0cf481804cdd5: 0.9 1.0 0.6 0.0
1a1fd87742c5ab42: 0.3 1.0 0.7 0.0
b91057cffaf55272: 0.0 1.0 0.6 0.0

6 users in cluser 1:
11891c406c088fdd: 0.5 0.1 1.0 0.0
ab2adf87b21afa43: 0.1 0.3 1.0 0.2
e976e6a0dd033ff4: 0.2 0.0 1.0 0.0
448a9dc88a199573: 0.0 0.3 1.0 0.0
0d2e63e0cad90a7d: 0.0 0.0 1.0 0.0
a7d3241a8cda5430: 0.0 0.0 0.9 0.0

16 users in cluser 2:
5ddbd530097789a